# Python: PySpark - Parquet

In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import lit

In [ ]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('parquet')
    .getOrCreate()
)

In [ ]:
# criando um df
dados = [
    ('Murilo', 'Oliveira', 'Brasileiro(a)', 'Engenheiro de Dados', 'M', 3000),
    ('Maria', 'Santana', 'Italiana', 'ADM', 'F', 3600),
    ('Paulo', 'Andrade', 'Francesa', 'Engenheiro de Dados', 'M', 3700),
    ('Ana', 'Clara', 'Portuguesa', 'Médico', 'F', 4000),
    ('Mario', 'Fernandes', 'Alemã', 'Jogador', 'M', 5000),
    ('Catarina', 'Silva', 'Brasileiro(a)', 'Advogado', 'F', 4500),
    ('Lucas', 'Pereira', 'Brasileiro(a)', 'Analista de Sistemas', 'M', 3800),
    ('Carolina', 'Costa', 'Portuguesa', 'Designer', 'F', 4200),
    ('Gabriel', 'Martins', 'Brasileiro(a)', 'Arquiteto', 'M', 4800),
    ('Larissa', 'Rocha', 'Brasileiro(a)', 'Enfermeira', 'F', 3700)
]

colunas = [
    'nome',
    'sobrenome',
    'nacionalidade',
    'trabalho',
    'genero',
    'salario'
]

df = spark.createDataFrame(dados, colunas)
df.show()

+--------+---------+-------------+--------------------+------+-------+
|    nome|sobrenome|nacionalidade|            trabalho|genero|salario|
+--------+---------+-------------+--------------------+------+-------+
|  Murilo| Oliveira|Brasileiro(a)| Engenheiro de Dados|     M|   3000|
|   Maria|  Santana|     Italiana|                 ADM|     F|   3600|
|   Paulo|  Andrade|     Francesa| Engenheiro de Dados|     M|   3700|
|     Ana|    Clara|   Portuguesa|              Médico|     F|   4000|
|   Mario|Fernandes|        Alemã|             Jogador|     M|   5000|
|Catarina|    Silva|Brasileiro(a)|            Advogado|     F|   4500|
|   Lucas|  Pereira|Brasileiro(a)|Analista de Sistemas|     M|   3800|
|Carolina|    Costa|   Portuguesa|            Designer|     F|   4200|
| Gabriel|  Martins|Brasileiro(a)|           Arquiteto|     M|   4800|
| Larissa|    Rocha|Brasileiro(a)|          Enfermeira|     F|   3700|
+--------+---------+-------------+--------------------+------+-------+



### Salvando o df no formato parquet

In [ ]:
df.write.parquet('/content/novo_df.parquet')

### Subscrevendo o arquivo parquet

In [ ]:
# permite atualizar o arquivo parquet
df = df.withColumn('status', lit('OK'))

df.write.mode('overwrite').parquet('/content/novo_df.parquet')

### Lendo o arquivo parquet

In [ ]:
df_parquet = spark.read.parquet('/content/novo_df.parquet')
df_parquet.show()

+--------+---------+-------------+--------------------+------+-------+------+
|    nome|sobrenome|nacionalidade|            trabalho|genero|salario|status|
+--------+---------+-------------+--------------------+------+-------+------+
|  Murilo| Oliveira|   Brasileiro| Engenheiro de Dados|     M|   3000|    OK|
|   Maria|  Santana|     Italiana|                 ADM|     F|   3600|    OK|
|   Paulo|  Andrade|     Francesa| Engenheiro de Dados|     M|   3700|    OK|
|     Ana|    Clara|   Portuguesa|              Médico|     F|   4000|    OK|
|   Mario|Fernandes|        Alemã|             Jogador|     M|   5000|    OK|
|Catarina|    Silva|   Brasileira|            Advogado|     F|   4500|    OK|
|   Lucas|  Pereira|   Brasileiro|Analista de Sistemas|     M|   3800|    OK|
|Carolina|    Costa|   Portuguesa|            Designer|     F|   4200|    OK|
| Gabriel|  Martins|   Brasileiro|           Arquiteto|     M|   4800|    OK|
| Larissa|    Rocha|   Brasileira|          Enfermeira|     F|  

### Particionando os dados parquet

In [ ]:
df.write\
.partitionBy('nacionalidade', 'genero')\
.mode('overwrite')\
.parquet('/content/novo_df.parquet')

#### Exibindo os dados do parquet cuja a nacionalidade é brasileiro(a)

In [ ]:
df_br = spark.read.parquet('/content/novo_df.parquet/nacionalidade=Brasileiro(a)')
df_br.show()

+--------+---------+--------------------+-------+------+
|    nome|sobrenome|            trabalho|salario|genero|
+--------+---------+--------------------+-------+------+
|  Murilo| Oliveira| Engenheiro de Dados|   3000|     M|
|   Lucas|  Pereira|Analista de Sistemas|   3800|     M|
| Gabriel|  Martins|           Arquiteto|   4800|     M|
|Catarina|    Silva|            Advogado|   4500|     F|
| Larissa|    Rocha|          Enfermeira|   3700|     F|
+--------+---------+--------------------+-------+------+



In [ ]:
# somente M
df_br_masc = spark.read.parquet('/content/novo_df.parquet/nacionalidade=Brasileiro(a)/genero=M')
df_br_masc.show()

+-------+---------+--------------------+-------+
|   nome|sobrenome|            trabalho|salario|
+-------+---------+--------------------+-------+
| Murilo| Oliveira| Engenheiro de Dados|   3000|
|  Lucas|  Pereira|Analista de Sistemas|   3800|
|Gabriel|  Martins|           Arquiteto|   4800|
+-------+---------+--------------------+-------+



In [ ]:
# somente F
df_br_fem = spark.read.parquet('/content/novo_df.parquet/nacionalidade=Brasileiro(a)/genero=F')
df_br_fem.show()

+--------+---------+----------+-------+
|    nome|sobrenome|  trabalho|salario|
+--------+---------+----------+-------+
|Catarina|    Silva|  Advogado|   4500|
| Larissa|    Rocha|Enfermeira|   3700|
+--------+---------+----------+-------+



### Realizando uma pesquisa via SQL no arquivo parquet particionado

In [ ]:
spark.sql('''
    CREATE OR REPLACE TEMPORARY VIEW funcionario
    USING parquet OPTIONS (path\"/content/novo_df.parquet/nacionalidade=Brasileiro(a)")
    '''
)

spark.sql('''
    SELECT *
    FROM funcionario
'''
).show()

+--------+---------+--------------------+-------+------+
|    nome|sobrenome|            trabalho|salario|genero|
+--------+---------+--------------------+-------+------+
|  Murilo| Oliveira| Engenheiro de Dados|   3000|     M|
|   Lucas|  Pereira|Analista de Sistemas|   3800|     M|
| Gabriel|  Martins|           Arquiteto|   4800|     M|
|Catarina|    Silva|            Advogado|   4500|     F|
| Larissa|    Rocha|          Enfermeira|   3700|     F|
+--------+---------+--------------------+-------+------+

